# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117808e+02     1.591249e+00
 * time: 0.7865450382232666
     1     1.035355e+01     8.078999e-01
 * time: 3.126711130142212
     2    -1.171575e+01     9.795259e-01
 * time: 3.775221109390259
     3    -3.390619e+01     7.183295e-01
 * time: 4.741167068481445
     4    -4.722593e+01     5.366174e-01
 * time: 5.678085088729858
     5    -5.681434e+01     2.068873e-01
 * time: 6.659291982650757
     6    -5.963103e+01     2.067385e-01
 * time: 7.287919998168945
     7    -6.081118e+01     1.143088e-01
 * time: 7.953727960586548
     8    -6.136379e+01     3.800089e-02
 * time: 8.594083070755005
     9    -6.164838e+01     3.106858e-02
 * time: 9.21233606338501
    10    -6.183710e+01     2.835819e-02
 * time: 9.87421202659607
    11    -6.198324e+01     1.982930e-02
 * time: 10.513626098632812
    12    -6.206414e+01     1.770723e-02
 * time: 11.146723985671997
    13    -6.210657e+01     1.326357e-02
 * time: 11.759362936019897
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557639
    AtomicNonlocal      14.8522605
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666459256
